# 2023 November Processing Template
Duncan

In [28]:
%load_ext autoreload
%autoreload 2

import pandas as pd
#import seaborn as sns

from htbam_analysis.processing import chip
from htbam_analysis.processing import experiment as exp
from htbam_analysis.processing import chipcollections as collections
from pathlib import Path 

%config InlineBackend.figure_format = 'svg'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Establish experiment and pinlist

In [2]:
root = '/path/to/experiment/directory/'
description = 'something_descriptive'
operator = 'whoami'
setup_num = 's#'
device_num = 'd#'
device_dimensions = (32, 56)
e = exp.Experiment(description, root, operator)

INFO     Experiment Initialized | Description: standard_curve_debugging, Operator: DFM


### Set Exposure and Channel Params

In [ ]:
button_quant_channel = 'egfp'
button_quant_exposure = 500
standard_channel = "PBP"
standard_exposure = 100
kinetics_channel = 'fura'
kinetics_exposure = 1000

## Mock a Pinlist for Flow-On Experiments

In [5]:
# Making a fake pinlist
#block 1 is the left side of the image, but right side of the device
block_descriptions = {1: 'protein_1', 2: 'protein_1', 3: 'protein_2', 4: 'protein_2'}
def get_block(c):
    return ((c // 8) + 1)
#creating a pin list
pinlist_dict = []
for c in range(32):
    for r in range(56):
        block = get_block(c)
        mutant = block_descriptions[block]
        pinlist_dict.append({'Indices': (c + 1, r + 1), 'MutantID': mutant})
pinlist_df = pd.DataFrame(pinlist_dict)
pinlist_path = f'{root}/data_analysis/20230208_pinlist.csv'
pinlist_df.to_csv(pinlist_path, index=False)

In [6]:
pinlist = e.read_pinlist(f'{root}/data_analysis/20230208_pinlist.csv')
pinlist.head(5)

Indices MutantID
x y                 
1 1  (1, 1)  Mac1_WT
  2  (1, 2)  Mac1_WT
  3  (1, 3)  Mac1_WT
  4  (1, 4)  Mac1_WT
  5  (1, 5)  Mac1_WT

## 2. Add devices and corners

In [13]:
# Top left, top right, bottom left, bottom right
d3_corners = ((452, 450),(6723,441),(472,6817),(6742,6800)) 

#Do this in imageJ, put your cursor over the reaction chambers and copy over
d3 = exp.Device(setup_num, device_num, device_dimensions, pinlist, d3_corners)


### Non-standard chamber-finding parameters

In [14]:
#chip.Stamp.circlePara1Index = 30
#chip.Stamp.circlePara2Index = 15

print('Old Chamber Radius: {} pixels'.format(chip.Stamp.chamberrad))
chip.Stamp.chamberrad = 32#33 in 2x2
print('New Chamber Radius: {} pixels'.format(chip.Stamp.chamberrad))

Old Chamber Radius: 32 pixels
New Chamber Radius: 32 pixels


## 3. Execute button analyses

#### Quantify egfp fluorescence on buttons

##### Load

In [6]:
# Stitched image path
button_quant_path = f'{root}/button_quant/20230401-095714-d2_egfp-quant_pre-MM_egfp_500_Sola_2x2_Kinetix-DynamicRange_4x/egfp/StitchedImages/BGSubtracted_StitchedImg_500_egfp_0.tif'

# Prepare ChipQuant object
d3_GFPQuant = collections.ChipQuant(d3, 'ButtonReference')

# Load image into memory
d3_GFPQuant.load_file(button_quant_path, button_quant_channel, button_quant_exposure)

##### Process

In [13]:
# Find buttons, extract intensity attributes
d3_GFPQuant.process()

# Summarize attributes as a pandas dataframe
quant_report1 = d3_GFPQuant.summarize()

Finding Buttons: 100%|██████████| 1792/1792 [03:04<00:00,  9.70it/s]


##### Save Summary/Image

In [14]:
quant_report1.to_csv(f'{root}/desired/file/name')

d3_GFPQuant.save_summary_image()

## 4. Standards

In [9]:
#define standard corners (if these are the same as your device above, you don't need to)
# Top left, top right, bottom left, bottom right
# new_corners = ((452, 450),(6723,441),(472,6817),(6742,6800))
d3_std_dev = exp.Device(setup_num, device_num, device_dimensions, pinlist, d3_corners)

In [41]:
# Create a standard series, load stitched images
standard_path = '/Volumes/T7/20221208_Mat2a_analysis.v2/20221202_tracy/standard_curve/Analysis_resized/dapi/StitchedImages'

d3_std = collections.StandardSeries(d3_std_dev, standard_channel)
d3_std.load_files(standard_path, standard_channel, standard_exposure)
d3_std.process(featuretype="chamber", coerce_center=False)

In [43]:
d3_std.save_summary()
d3_std.save_summary_images()

## 5. Chamber Reference

In [15]:
#code is having a hard time finding chambers in the standard curve (perhaps because it was resized?), so using this image as a reference

d3_ChamberRef = collections.ChipQuant(d3, 'Chamber_Ref')
chamber_ref = '/Users/duncanmuir/Desktop/20231114_std_curve_test_data_duncan/standard/Analysis/StitchedImg_1000_5_500.tif'
d3_ChamberRef.load_file(chamber_ref, 'fura', 1000)

In [16]:
d3_ChamberRef.process(mapped_features = 'chamber', coerce_center = False)


In [17]:
d3_ChamberRef.save_summary_image(feature_type = 'chamber')


### 6. Kinetics

In [10]:
# Assay Series Descriptions & Object
parent = Path('/Volumes/DuncanSSD/20230331/kinetics/')

kinetic_descriptions = ['7_8125uM_ADP',
                        # '15_6125uM_ADP',
                        # '31_25uM_ADP',
                        # '62_5uM_ADP',
                        # '125uM_ADP',
                        # '250uM_ADP',
                        # '500uM_ADP',
                        # '1000uM_ADP',
                        ]

# References (for positions)
chamberRef = d3_ChamberRef.chip
buttonRef = d3_GFPQuant.chip

# the name of your button quant files
gfp_descriptions = ['egfp-quant_pre-MM']

kinetic_file_handles = kinetic_descriptions
gfp_file_handles = gfp_descriptions

adk_orth = collections.AssaySeries(d3, 
                                kinetic_descriptions, 
                                chamberRef, 
                                buttonRef)

In [12]:
kin_root = '/Volumes/DuncanSSD/20230331/kinetics/'

adk_orth.parse_kineticsFolders(kin_root, kinetic_file_handles, kinetic_descriptions, kinetics_channel, kinetics_exposure)



In [13]:
button_root = '/Volumes/DuncanSSD/20230331/button_quant/'

adk_orth.parse_quantificationFolders(button_root, gfp_descriptions, gfp_file_handles, button_quant_channel, button_quant_exposure)



In [14]:
adk_orth.process_kinetics(low_mem = False)

Series <7_8125uM-ADP> Stamped and Mapped: 100%|██████████| 20/20 [00:39<00:00,  1.96s/it]


In [35]:
adk_orth.process_quants()

Mapping and Processing Buttons: 100%|██████████| 12/12 [00:50<00:00,  4.20s/it]


In [36]:
adk_orth.save_summary(outPath = kin_root)